In [1]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 

In [2]:
#connect to postgres database 
conn = psycopg2.connect(database="donniedata", user=config.db_user, password=config.db_password, sslmode="disable")
cur = conn.cursor();
conn.autocommit = True

In [3]:
#get sample of data -1000 rows
endpoint = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=1000"
r = requests.get(url=endpoint)

In [295]:
data = r.json()

In [297]:
data[0:2]

[{'transmission_datetime': '196158860_4_06092020073616',
  'post_id': '700-14150',
  'street_block': 'VALENCIA ST 1400',
  'payment_type': 'CASH',
  'session_start_dt': '2020-06-09T07:36:16.000',
  'session_end_dt': '2020-06-09T10:36:00.000',
  'meter_event_type': 'NS',
  'gross_paid_amt': '0.8'},
 {'transmission_datetime': '196158861_4_06092020073722',
  'post_id': '471-07180',
  'street_block': 'HARRISON ST 700',
  'payment_type': 'CREDIT CARD',
  'session_start_dt': '2020-06-09T07:37:22.000',
  'session_end_dt': '2020-06-09T17:30:10.000',
  'meter_event_type': 'NS',
  'gross_paid_amt': '4.25'}]

## payment table 

In [299]:
#insert data into street dimension
sql = '''
DO $$
BEGIN 
IF NOT EXISTS (select 1 from sf_ticket_trans.dim_payment where payment_type = %(payment)s) 
THEN INSERT INTO sf_ticket_trans.dim_payment (payment_type) VALUES (%(payment)s);
END IF;
END;
$$ 

'''

#refactored execution option to irate through json object itself 
[cur.execute(sql, {'payment':data[i]['payment_type']}) for i in list(range(len(data)))];
    

In [302]:
#data inserstion test succesful wiht only unqiue values being added
cur.execute('select count(*) from sf_ticket_trans.dim_payment')
print('Unique values inserted:', cur.fetchall()[0][0])

payment_type_list = [data[i]['payment_type'] for i in list(range(len(data)))]
print('Unique values from data pull:', len(set(payment_type_list)))

Unique values inserted: 3
Unique values from data pull: 3


## Streets data 

In [277]:
#create street list 
street_list = [data[i]['street_block'] for i in list(range(len(data)))]

In [279]:
#insert data into street dimension
sql = '''
DO $$
BEGIN 
IF NOT EXISTS (select 1 from sf_ticket_trans.dim_street where street_block = %(street)s) 
THEN INSERT INTO sf_ticket_trans.dim_street (street_block) VALUES ( %(street)s  );
END IF;
END;
$$ 

'''
    
[cur.execute(sql, {'street':i}) for i in street_list];
    

In [285]:
#data inserstion test succesful wiht only unqiue values being added
cur.execute('select count(*) from sf_ticket_trans.dim_street')
print('Unique values inserted:', cur.fetchall()[0][0])
print('Unique values from data pull:', len(set(street_list)))

Unique values inserted: 491
Unique values from data pull: 491


## fact table 

In [321]:
#query to get street_id and insert 
#subquery to select id where value= lookup-value and insert with new data
sql ='''

INSERT INTO sf_ticket_trans.fact_transactions (
    transmission_datetime ,
    payment_type_id,
    street_block_id ,
    post_id ,
    meter_event_type ,
    gross_paid_amt ,
    session_start_dt ,
    session_end_date 
    )
VALUES (
    %(transmission_dt)s ,
    (SELECT payment_type_id 
        FROM sf_ticket_trans.dim_payment WHERE payment_type = %(payment)s ) ,
    (SELECT street_block_id 
        FROM sf_ticket_trans.dim_street WHERE street_block = %(street)s ) ,
    %(post)s ,
    %(meter_event)s ,
    %(paid_amt)s ,
    %(sessionstart)s ,
    %(sessionend)s 
    );

'''
#iterate through json object and insert into fact table 
for i in list(range(len(data))): 
    cur.execute(sql, {
                     'transmission_dt' : data[i]['transmission_datetime'],
                      'payment': data[i]['payment_type'],
                      'street': data[i]['street_block'],
                     'post' : data[i]['post_id'],
                      'meter_event': data[i]['meter_event_type'],
                      'paid_amt': data[i]['gross_paid_amt'],
                     'sessionstart' : data[i]['session_start_dt'],
                      'sessionend': data[i]['session_end_dt']
                    })
    
    
                

## validate 

In [322]:
data_df = pd.DataFrame(data)

In [334]:
round(len(data_df) * 0.25)

250

In [478]:
#get sample of subset 
sample_size = round(len(data_df) * 0.25)
sample_ids = data_df.sample(sample_size)['transmission_datetime'].to_list()
sample_id_tuple = tuple(sample_ids)

In [365]:
sample_df = data_df[data_df['transmission_datetime'].isin(sample_ids)][['transmission_datetime','payment_type','street_block']]
sample_df.head()

,transmission_datetime,payment_type,street_block
7,196158867_4_06092020073841,CREDIT CARD,SPEAR ST 400
8,196158868_4_06092020073825,CREDIT CARD,EMBARCADERO NORTH 300
9,196158869_4_06092020073758,CASH,BROADWAY 0
11,196158871_4_06092020073928,CREDIT CARD,07TH ST 200
13,196158873_4_06092020073931,CREDIT CARD,SPEAR ST 0


In [481]:
sql = (''' 
SELECT tran.transmission_datetime,
        pay.payment_type,
        str.street_block
FROM sf_ticket_trans.fact_transactions tran
    LEFT JOIN sf_ticket_trans.dim_payment pay
        ON tran.payment_type_id = pay.payment_type_id
    LEFT JOIN sf_ticket_trans.dim_street str
        ON tran.street_block_id = str.street_block_id
WHERE tran.transmission_datetime IN %s
''')

#value tuple must also be 'tupled' psycopg2 expects a sequence of sequence  
cur.execute(sql, (sample_id_tuple,))
result_df = pd.DataFrame(cur.fetchall())
result_df.columns = [i[0] for i in cur.description]

In [482]:
result_df

,transmission_datetime,payment_type,street_block
0,196158863_4_06092020073706,CASH,SANSOME ST 300
1,196158865_4_06092020073802,CREDIT CARD,SPEAR ST 300
2,196158869_4_06092020073758,CASH,BROADWAY 0
3,196158886_4_06092020074050,CREDIT CARD,PINE ST 300
4,196158889_4_06092020074102,CREDIT CARD,EMBARCADERO SOUTH 400
...,...,...,...
245,196159864_4_06092020085155,CREDIT CARD,07TH ST 1300
246,196159866_4_06092020085213,CREDIT CARD,OFARRELL ST 2200
247,196159872_4_06092020085133,CASH,MISSION ST 2000
248,196159873_4_06092020085230,CREDIT CARD,SACRAMENTO ST 2400
